In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt

数据集准备

In [2]:
inputs  = []
outputs = []

d = {}
d['Iris-setosa\n']     = [1., 0., 0.]
d['Iris-versicolor\n'] = [0., 1., 0.]
d['Iris-virginica\n']  = [0., 0., 1.]

with open('iris.data') as f:
    lines=f.readlines()
    for line in lines:
        items=line.split(',')
        if len(items) == 5:
            inp =  [float(x) for x in items[0:4] ]
            inputs.append(inp)
            out = d[items[4]]
            outputs.append(out)
            
ids=random.sample(range(0,len(inputs)), 100) # generate 100 random ids
#training set
train_in = []
train_out=[]
for id in ids:
    train_in.append(inputs[id])
    train_out.append(outputs[id])
train_inputs  = np.array(train_in)
train_outputs = np.array(train_out)

#valid set
val_input =[]
val_output=[]
validation_ids = list(set(range(0,len(inputs))) - set(ids))
for val_id in validation_ids:
    val_input.append(inputs[val_id])
    val_output.append(outputs[val_id])
val_inputs  = np.array(val_input)
val_outputs = np.array(val_output)

val_dataset_inputs = []
val_dataset_inputs = []
for i,val_input in enumerate(val_inputs):
  val_dataset_inputs.append(np.append(val_input,[1]))
print(val_dataset_inputs[10:15])

[array([5.7, 4.4, 1.5, 0.4, 1. ]), array([5.4, 3.9, 1.3, 0.4, 1. ]), array([5.1, 3.5, 1.4, 0.3, 1. ]), array([5.7, 3.8, 1.7, 0.3, 1. ]), array([6.7, 3.3, 5.7, 2.5, 1. ])]


通过上一个博客我们可以得到单一的感知器可以判断是否为某种类型的鸢尾花，通过计算我们可以得到已下三个%\vec w%分别代表不同的已训练感知器以及其精度：


In [3]:
# Setosa accuracy：1.0
w0 = [-3.59667091,9.75509188,-1.33161528,-2.25896886,-3.00288717]
# Versicolor accuracy：0.8
w1 = [0.05378028,-3.4590448,1.10002562,-2.89421222,9.10068046]
# Virginica accuracy：0.88
w2 = [-4.89257798,1.67271247,5.2495293,-1.78437464,2.81813974]

w_v = []
w_v.append(w0)
w_v.append(w1)
w_v.append(w2)

现在我们将让其再次经过一个人工逻辑编写的神经元，是否能进一步提升accuracy呢？

In [4]:
def artificial_neutral(unite_result):
    if unite_result[0]==1: # setosa perceptron has 100% accuracy
        return [1,0,0]   # so it always is corrected 
    elif unite_result[0]==0 and unite_result[2]==0: 
        return [0,1,0]   #virginica has higer accuracy then versicolor 
    elif unite_result[2]==1:# so use its estimation
        return [0,0,1]
    else:
        return [0,1,0] # final estimate only is versicolor 

获取第一层感知器结果：

In [5]:
w_v = np.array(w_v)
print(len(val_dataset_inputs))
l1_unite_result_list = []
for input_data in val_dataset_inputs:
    l1_unite_result = []
    for w in w_v:
        if input_data.dot(w) > 0:
            l1_unite_result.append(1)
        else:
            l1_unite_result.append(0)
    l1_unite_result_list.append(l1_unite_result)

50


获取到最终结果：

In [6]:
corrected_counter = 0
for i,second_layer_input in enumerate(l1_unite_result_list):
  layer2_result = artificial_neutral(second_layer_input)
  if (val_outputs[i] == layer2_result).all():
    corrected_counter+=1
print("second layer neutral accuracy :{}".format(corrected_counter/len(val_dataset_inputs)))

second layer neutral accuracy :0.92


现在我们输入数据就可以直接拿到鸢尾花的种类，现在我们看看判断变色鸢尾的概率是否提升

首先我们准备一些变色鸢尾的数据：

In [7]:
versicolor_inputs = []
versicolor_output = np.array([0., 1., 0.])
for i,val_output in enumerate(val_outputs):
    if (val_output == versicolor_output).all():
        versicolor_inputs.append(val_dataset_inputs[i])
print(len(versicolor_inputs))


12


In [8]:
w_v = np.array(w_v)
print(len(versicolor_inputs))
l1_unite_result_list = []
for input_data in versicolor_inputs:
    l1_unite_result = []
    for w in w_v:
        if input_data.dot(w) > 0:
            l1_unite_result.append(1)
        else:
            l1_unite_result.append(0)
    l1_unite_result_list.append(l1_unite_result)

corrected_counter = 0
for i,second_layer_input in enumerate(l1_unite_result_list):
  layer2_result = artificial_neutral(second_layer_input)
  if (versicolor_output == layer2_result).all():
    corrected_counter+=1
print("second layer neutral accuracy for versicolor :{}".format(corrected_counter/len(versicolor_inputs)))

12
second layer neutral accuracy :0.9166666666666666


我们可以看到现在对于变色鸢尾的判断准确性大大提高了